In [1]:
import pandas as pd
import os
import numpy as np
import re
from pathlib import Path
import glob

In [2]:
current_directory = Path.cwd()
parent_dir = current_directory.parent

In [3]:
survey_years = ['2013', '2015', '2017', '2019', '2022']

In [4]:
all_dfs = {}

for year in survey_years:
    # Use the filename (or a derivative) as the key
    PCS_data_folder_path = f'{parent_dir}/data'

    file_name = f'{PCS_data_folder_path}/Patient_Characteristics_Survey__PCS___{year}.csv'
    # Load the dataframe and assign it to the dictionary
    
    all_dfs[file_name] = pd.read_csv(file_name)

    new_file_name = f'PCS{year}'

    all_dfs[new_file_name] = all_dfs.pop(file_name)


In [5]:
class ColumnDifferences:
    
    def __init__(self, all_dfs):
    
        self.original_column_names = {}

        for df in all_dfs.keys():
            self.original_column_names[f'{df}_column_names'] = list(all_dfs[df].columns)

        self.in_other_but_not_in_2022 = {}
        self.in_2022_but_not_in_other = {}
        self.similarities_in_column_names = {}

        for column_names in self.original_column_names.keys():

            if column_names == 'PCS2022_column_names':
                break

            year = column_names[3:7]

            column_names_2022 = set(self.original_column_names['PCS2022_column_names'])
            other_column_names = set(self.original_column_names[column_names])
            # Finds column names in the other dataframes that are not in the 2022 dataframe
            self.in_other_but_not_in_2022[f'In {year} but not in 2022'] = other_column_names - column_names_2022
            self.in_2022_but_not_in_other[f'In 2022 but not in {year}'] = column_names_2022 - other_column_names
            self.similarities_in_column_names[f'In 2022 & {year}'] = column_names_2022.intersection(other_column_names)

In [6]:
diffs = ColumnDifferences(all_dfs)

print(diffs.in_other_but_not_in_2022)
print(diffs.in_2022_but_not_in_other)

{'In 2013 but not in 2022': {'Number of Hours Worked Each Week', 'Family Health Plus Insurance', 'No Chronic Med. Condition', 'Receives Smoking Medication', 'Drug Substance Related Disorder', 'Primary Language', 'Unknown Chronic Med. Condition', 'Hearing Visual Impairment', 'Other Developmental Disabilities', 'Educational Status', 'Pulmonary/Asthma', 'Receives Smoking Counseling', 'Three Digit Residential Zip Code', 'Special Educational Services'}, 'In 2015 but not in 2022': {'Hearing Visual Impairment'}, 'In 2017 but not in 2022': {'Hearing Visual Impairment'}, 'In 2019 but not in 2022': set()}
{'In 2022 but not in 2013': {'Other Developmental Disability', 'Drug Substance Disorder', 'Visual Impairment', 'Alcohol 12m Service', 'Number Of Hours Worked Each Week', 'Drug/Substance 12m Service', 'No Chronic Med Condition', 'Unknown Chronic Med Condition', 'Opioid Related Disorder', 'Pulmonary Asthma', 'Other Chronic Med Condition', 'Preferred Language', 'Opioid 12m Service', 'Hearing Impai

In [7]:
columns_to_drop = ['Alcohol 12m Service', 'Cannabis Medicinal Use', 'Cannabis Recreational Use', 'Drug/Substance 12m Service', 'Opioid 12m Service', 
                   'Opioid Related Disorder', 'Religious Preference', 'Speech Impairment', 'Hearing Impairment', 'Visual Impairment', 'Hearing Visual Impairment',
                   'Family Health Plus Insurance', 'Other Chronic Med Condition']

columns_to_rename = {'Educational Status': 'Education Status', 
                     'Drug Substance Related Disorder': 'Drug Substance Disorder', 
                     'No Chronic Med. Condition': 'No Chronic Med Condition',
                     'Other Developmental Disabilities': 'Other Developmental Disability',
                     'Number of Hours Worked Each Week': 'Number Of Hours Worked Each Week',
                     'Primary Language': 'Preferred Language',
                     'Pulmonary/Asthma': 'Pulmonary Asthma',
                     'Receives Smoking Counseling': 'Received Smoking Counseling',
                     'Receives Smoking Medication': 'Received Smoking Medication',
                     'Special Educational Services': 'Special Education Services',
                     'Three Digit Residential Zip Code': 'Three Digit Residence Zip Code',
                     'Unknown Chronic Med. Condition': 'Unknown Chronic Med Condition'}

In [8]:
for year, df in all_dfs.items():
    # Rename columns
    for name, rename in columns_to_rename.items():
        if name in df.columns:
            all_dfs[year] = all_dfs[year].rename(columns={name: rename})

    # Drop columns
    all_dfs[year] = all_dfs[year].drop(
        columns=[col for col in columns_to_drop if col in df.columns])

In [9]:
diffs = ColumnDifferences(all_dfs)

print(diffs.in_other_but_not_in_2022)
print(diffs.in_2022_but_not_in_other)

{'In 2013 but not in 2022': set(), 'In 2015 but not in 2022': set(), 'In 2017 but not in 2022': set(), 'In 2019 but not in 2022': set()}
{'In 2022 but not in 2013': set(), 'In 2022 but not in 2015': set(), 'In 2022 but not in 2017': set(), 'In 2022 but not in 2019': set()}


In [10]:
class ValueDifferences:
    
    def __init__(self, all_dfs, canonical_year='PCS2022'):
        
        self.original_values = {}
        for year in all_dfs.keys():
            unique_values = {}
            for column in all_dfs[year].columns:
                unique_values[column] = [str(val) for val in all_dfs[year][column].unique()]
            self.original_values[year] = unique_values

        self.in_other_but_not_in_2022 = {}
        self.in_2022_but_not_in_other = {}
        self.in_2022_and_other = {}

        for year in self.original_values.keys():
            if year == canonical_year:
                continue

            self.in_other_but_not_in_2022[year] = {}
            self.in_2022_but_not_in_other[year] = {}
            self.in_2022_and_other[year] = {}

            for col_name in self.original_values[year].keys():
                if col_name not in self.original_values[canonical_year]:
                    continue

                values_2022 = set(self.original_values[canonical_year][col_name])
                values_other = set(self.original_values[year][col_name])

                self.in_other_but_not_in_2022[year][col_name] = list(values_other - values_2022)
                self.in_2022_but_not_in_other[year][col_name] = list(values_2022 - values_other)
                self.in_2022_and_other[year][col_name] = list(values_2022 & values_other)

        # Remove empty entries
        self.in_other_but_not_in_2022 = self._remove_empty(self.in_other_but_not_in_2022)
        self.in_2022_but_not_in_other = self._remove_empty(self.in_2022_but_not_in_other)
        self.in_2022_and_other = self._remove_empty(self.in_2022_and_other)

    @staticmethod
    def _remove_empty(d):
        return {year: {col: vals for col, vals in cols.items() if vals}
                for year, cols in d.items()}

In [11]:
val_diffs = ValueDifferences(all_dfs)

# Specific year and column
print(val_diffs.in_other_but_not_in_2022)
print(val_diffs.in_2022_but_not_in_other)


{'PCS2013': {'Survey Year': ['2013'], 'Age Group': ['UNKNOWN'], 'Sexual Orientation': ['LESBIAN, GAY, OR HOMOSEXUAL'], 'Principal Diagnosis Class': ['ORGANIC MENTAL DISORDER', 'MENTAL RETARDATION/DEV. DISORDER', 'UNKNOWN/DEFERRED', 'SUBSTANCE-RELATED DISORDER', 'PHYSICAL DISORDER'], 'Additional Diagnosis Class': ['ORGANIC MENTAL DISORDER', 'MENTAL RETARDATION/DEV. DISORDER', 'UNKNOWN/DEFERRED', 'SUBSTANCE-RELATED DISORDER', 'PHYSICAL DISORDER']}, 'PCS2015': {'Survey Year': ['2015'], 'Age Group': ['UNKNOWN'], 'Hispanic Ethnicity': ['YES']}, 'PCS2017': {'Survey Year': ['2017'], 'Age Group': ['UNKNOWN']}, 'PCS2019': {'Survey Year': ['2019'], 'Age Group': ['UNKNOWN'], 'Unknown Chronic Med Condition': ['False', 'True'], 'Unknown Insurance Coverage': ['False', 'True']}}
{'PCS2013': {'Survey Year': ['2022'], 'Sex': ['X (NON-BINARY)'], 'Transgender': ["CLIENT DIDN'T ANSWER"], 'Sexual Orientation': ['LESBIAN OR GAY', 'CLIENT DID NOT ANSWER'], 'Received Smoking Medication': ['UNKNOWN'], 'Receive

In [12]:
rename_values = {'Sexual Orientation': {'LESBIAN, GAY, OR HOMOSEXUAL': 'LESBIAN OR GAY',
                                        'CLIENT DID NOT ANSWER': 'UNKNOWN'},
                 'Transgender': {"CLIENT DIDN'T ANSWER": 'UNKNOWN'},
                 'Principal Diagnosis Class': {'UNKNOWN/DEFERRED': 'UNKNOWN',
                                                'MENTAL RETARDATION/DEV. DISORDER': 'NOT MI - DEVELOPMENTAL DISORDERS',
                                                'SUBSTANCE-RELATED DISORDER': 'SUBSTANCE-RELATED AND ADDICTIVE DISORDERS',
                                                'ORGANIC MENTAL DISORDER': 'NOT MI - ORGANIC MENTAL DISORDER',
                                                'PHYSICAL DISORDER': 'NOT MI - OTHER'},
                'Additional Diagnosis Class': {'UNKNOWN/DEFERRED': 'UNKNOWN',
                                               'SUBSTANCE-RELATED DISORDER': 'SUBSTANCE-RELATED AND ADDICTIVE DISORDERS',
                                               'ORGANIC MENTAL DISORDER': 'NOT MI - ORGANIC MENTAL DISORDER',
                                               'MENTAL RETARDATION/DEV. DISORDER': 'NOT MI - DEVELOPMENTAL DISORDERS',
                                               'PHYSICAL DISORDER': 'NOT MI - OTHER'},
                'Hispanic Ethnicity': {'YES': 'YES, HISPANIC/LATINO'},
                'Unknown Chronic Med Condition': {True: 'YES',
                                                  False: 'NO'},
                'Unknown Insurance Coverage': {True: 'YES',
                                               False: 'NO'},
                'Sex': {'X (NON-BINARY)': 'UNKNOWN'},
                'Race': {'UNKNOWN RACE': 'UNKNOWN'},
                'Employment Status': {'UNKNOWN EMPLOYMENT STATUS': 'UNKNOWN'},
                'Number Of Hours Worked Each Week': {'UNKNOWN EMPLOYMENT HOURS': 'UNKNOWN'}}

drop_values = {'Received Smoking Medication': {'UNKNOWN'},
               'Received Smoking Counseling': {'UNKNOWN'},
               'Serious Mental Illness': {'UNKNOWN'},
               'Additional Diagnosis Class': {'NO ADDITIONAL DIAGNOSIS'},
               'Age Group': {'UNKNOWN'}}

In [13]:
for year, df in all_dfs.items():
    for col, mapping in rename_values.items(): # Rename values to align with 2022 schema
        if col in df.columns:
            all_dfs[year][col] = all_dfs[year][col].replace(mapping)

    for col, values_to_drop in drop_values.items(): # Drop rows containing specific values
        if col in df.columns:
            all_dfs[year] = all_dfs[year][~all_dfs[year][col].isin(values_to_drop)]
    
    all_dfs[year] = all_dfs[year].replace(['NOT APPLICABLE', 'UNKNOWN'], np.nan)

    all_dfs[year].loc[all_dfs[year]['Employment Status'].isin(
        ['NOT IN LABOR FORCE:UNEMPLOYED AND NOT LOOKING FOR WORK', 'UNEMPLOYED, LOOKING FOR WORK']),
        'Number Of Hours Worked Each Week'] = 'UNEMPLOYED/NOT IN LABOR FORCE'

In [14]:
# Validate that renames worked
val_diffs = ValueDifferences(all_dfs)
print(val_diffs.in_other_but_not_in_2022)
print(val_diffs.in_2022_but_not_in_other)

{'PCS2013': {'Survey Year': ['2013']}, 'PCS2015': {'Survey Year': ['2015']}, 'PCS2017': {'Survey Year': ['2017']}, 'PCS2019': {'Survey Year': ['2019']}}
{'PCS2013': {'Survey Year': ['2022'], 'Three Digit Residence Zip Code': ['777']}, 'PCS2015': {'Survey Year': ['2022'], 'Three Digit Residence Zip Code': ['777']}, 'PCS2017': {'Survey Year': ['2022'], 'Three Digit Residence Zip Code': ['777']}, 'PCS2019': {'Survey Year': ['2022']}}


In [15]:
file_path = f'{parent_dir}/outputs/data/categorical'
os.makedirs(file_path, exist_ok=True)

for year in survey_years:    
    all_dfs[f'PCS{year}'].to_csv(f'{file_path}/PCS{year}.csv', index=False)

PCS_combined = pd.concat(all_dfs.values(), axis=0, ignore_index=True)
PCS_combined.to_csv(f'{file_path}/Patient Characteristics Survey (Years 2013 - 2022) (categorical).csv', 
                     index=False, encoding='utf-8-sig')

In [16]:
cols_to_ignore = ['Survey Year', 'Three Digit Residence Zip Code']

categorical_mappings = {}
for col in all_dfs['PCS2022'].columns:
    if col in cols_to_ignore:
        continue
    # Filter out NaN values before creating mappings
    unique_vals = sorted([val for val in all_dfs['PCS2022'][col].unique() if pd.notna(val)], 
                         key=str)
    categorical_mappings[col] = {val: i for i, val in enumerate(unique_vals)}

# Apply mappings to all dataframes
for year, df in all_dfs.items():
    for col, mapping in categorical_mappings.items():
        if col in df.columns:
            # map() preserves NaN for values not in the mapping
            all_dfs[year][col] = all_dfs[year][col].map(mapping)

# Create data dictionary
with open(f'{parent_dir}/outputs/data/numerical/data_dictionary.txt', 'w') as f:
    for col, mapping in categorical_mappings.items():
        f.write(f'{col}:\n')
        for original_value, encoded_value in sorted(mapping.items(), key=lambda x: x[1]):
            f.write(f'  {encoded_value}: {original_value}\n')
        f.write('\n')

In [17]:
file_path = f'{parent_dir}/outputs/data/numerical'
os.makedirs(file_path, exist_ok=True)

for year in survey_years:    
    all_dfs[f'PCS{year}'].to_csv(f'{file_path}/PCS{year}.csv', index=False)

PCS_combined = pd.concat(all_dfs.values(), axis=0, ignore_index=True)
PCS_combined.to_csv(f'{file_path}/Patient Characteristics Survey (Years 2013 - 2022) (numerical).csv', 
                     index=False, encoding='utf-8-sig')